In [139]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df
from project_modules.classifcation import classify_MP,getXY, boruta_fs
from project_modules.utils import MPutils
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import cross_val_score
import cupy as cp
from datetime import datetime

from tqdm.notebook import tqdm
from copy import deepcopy

# from project_modules.utils import get_logger
# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

pd.options.display.max_columns = None
pd.options.display.max_rows = 50
pd.options.display.max_colwidth = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [145]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]
lResCol = [
    "Title",
    "cv",
    "param_clf",
    "param_clf__max_depth",
    "param_clf__n_estimators",
    "param_clf__random_state",
    # "param_clf__max_iter",
    "mean_test_accuracy",
    "mean_test_balanced_accuracy",
    "mean_test_roc_auc",
    "mean_test_f1",
    "mean_test_recall",
    "mean_test_sensitivity",
    "mean_test_specificity",
    "mean_test_precision",
    "mean_test_NPV",
    "mean_test_PPV",
]

In [146]:
site_name_dict = {1: "LDN", 2: "MTL", 3: "SAN", 4: "RIO", 5: "LUS", 6: "CA", 7: "NA"}
status_name_dict = {
    0: "LC_NEG",
    1: "LC_POS",
    2: "HC",
}

# Combine Results

In [147]:
import glob

path = "OUTPUT/MP/05-classifiers/DataV4"
files = glob.glob(path + '/*/CA*.csv')
# files = files+glob.glob(path + "/*/FS*.csv")
files = files + glob.glob(path + "/*/Val*.csv")
files = files+glob.glob(path+'/*/CLFRun*.csv')
files = [x for x in files if 'selcol' not in x.lower()]

print(files)
print(len(files))

['OUTPUT/MP/05-classifiers/DataV4/dfcmpltPreLC4_SITE4/CA_dfcmpltPreLC4_SITE4_Full.csv', 'OUTPUT/MP/05-classifiers/DataV4/dfcmpltPreLC4_SITE4/CA_dfcmpltPreLC4_SITE4_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV4/dfcmpltPreLC4_SITE1/CA_dfcmpltPreLC4_SITE1_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV4/dfcmpltPreLC4_SITE1/CA_dfcmpltPreLC4_SITE1_Full.csv', 'OUTPUT/MP/05-classifiers/DataV4/dfcmpltPreLC4_SITE3/CA_dfcmpltPreLC4_SITE3_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV4/dfcmpltPreLC4_SITE3/CA_dfcmpltPreLC4_SITE3_Full.csv', 'OUTPUT/MP/05-classifiers/DataV4/dfcmpltPreLC4/CA_dfcmpltPreLC4_Full.csv', 'OUTPUT/MP/05-classifiers/DataV4/dfcmpltPreLC4/CA_dfcmpltPreLC4_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV4/dfcmpltPreLC4_SITE2/CA_dfcmpltPreLC4_SITE2_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV4/dfcmpltPreLC4_SITE2/CA_dfcmpltPreLC4_SITE2_Full.csv', 'OUTPUT/MP/05-classifiers/DataV4/dfcmpltPreLC4_SITE1/CLFRun_dfcmpltPreLC4_SITE1_Results.csv', 'OUTPUT/MP/05-classifiers/DataV4/dfcmpltPreL

In [148]:
ldf = []
for i in files:
    ldf.append(pd.read_csv(i))

In [149]:
dfRes = pd.concat(ldf)

In [150]:
def changeSite(x):
    if pd.isnull(x):
        return np.nan
    if 'SITE7' in x:
        return 'NorthAmerica'
    if 'SITE6' in x:
        return 'Canada'
    if 'SITE5' in x:
        return 'LUS'
    if 'SITE4' in x:
        return 'RIO'
    if 'SITE3' in x:
        return 'SAN'
    if 'SITE2' in x:
        return 'MTL'
    if 'SITE1' in x:
        return 'LDN'
    return "Global"

In [151]:
dfRes

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_balanced_accuracy,split1_test_balanced_accuracy,split2_test_balanced_accuracy,split3_test_balanced_accuracy,split4_test_balanced_accuracy,mean_test_balanced_accuracy,std_test_balanced_accuracy,rank_test_balanced_accuracy,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_sensitivity,split1_test_sensitivity,split2_test_sensitivity,split3_test_sensitivity,split4_test_sensitivity,mean_test_sensitivity,std_test_sensitivity,rank_test_sensitivity,split0_test_specificity,split1_test_specificity,split2_test_specificity,split3_test_specificity,split4_test_specificity,mean_test_specificity,std_test_specificity,rank_test_specificity,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_NPV,split1_test_NPV,split2_test_NPV,split3_test_NPV,split4_test_NPV,mean_test_NPV,std_test_NPV,rank_test_NPV,split0_test_PPV,split1_test_PPV,split2_test_PPV,split3_test_PPV,split4_test_PPV,mean_test_PPV,std_test_PPV,rank_test_PPV,cv,Title,RunType,MainDataset,date,brt_nTrue,brt_nTop,brt_md,brt_params
0,0.103384,0.014312,0.023572,0.002065,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_depth': 3, 'clf__n_estimators': 100, 'clf__random_state': 42}",0.954248,0.993464,0.967105,0.973684,0.967105,0.971121,0.012827,41,0.860000,0.980000,0.900000,0.920000,0.900000,0.912000,0.039192,41,1.000000,0.999687,0.993386,1.000000,0.994961,0.997607,0.002850,32,0.973384,0.996109,0.980695,0.984496,0.980695,0.983076,0.007445,41,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1,0.720000,0.960000,0.800000,0.840000,0.800000,0.824000,0.078384,41,0.948148,0.992248,0.962121,0.969466,0.962121,0.966821,0.014466,41,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1,0.948148,0.992248,0.962121,0.969466,0.962121,0.966821,0.014466,41,5,dfcmpltPreLC4_SITE4_Full,Full,dfcmpltPreLC4_SITE4,2024-10-09 02:30:52.157500,NaN,NaN,NaN,NaN
1,1.048602,0.126276,0.091691,0.020825,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_depth': 3, 'clf__n_estimators': 1000, 'clf__random_state': 42}",0.954248,0.993464,0.967105,0.973684,0.973684,0.972437,0.012685,40,0.860000,0.980000,0.900000,0.920000,0.920000,0.916000,0.038781,40,1.000000,1.000000,0.992756,1.000000,0.995591,0.997669,0.002992,31,0.973384,0.996109,0.980695,0.984496,0.984496,0.983836,0.007357,40,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1,0.720000,0.960000,0.800000,0.840000,0.840000,0.832000,0.077563,40,0.948148,0.992248,0.962121,0.969466,0.969466,0.968290,0.014286,40,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1,0.948148,0.992248,0.962121,0.969466,0.969466,0.968290,0.014286,40,5,dfcmpltPreLC4_SITE4_Full,Full,dfcmpltPreLC4_SITE4,2024-10-09 02:30:52.157500,NaN,NaN,NaN,NaN
2,0.113176,0.011183,0.025058,0.000859,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_depth': 6, 'clf__n_estimators': 100, 'clf__random_state': 42}",0.954248,0.993464,0.980263,0.973684,0.980263,0.976385,0.012799,38

In [152]:
dfRes.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_clf', 'param_clf__max_depth', 'param_clf__n_estimators',
       'param_clf__random_state', 'param_clf__max_iter',
       'param_clf__learning_rate', 'params', 'split0_test_accuracy',
       'split1_test_accuracy', 'split2_test_accuracy', 'split3_test_accuracy',
       'split4_test_accuracy', 'mean_test_accuracy', 'std_test_accuracy',
       'rank_test_accuracy', 'split0_test_balanced_accuracy',
       'split1_test_balanced_accuracy', 'split2_test_balanced_accuracy',
       'split3_test_balanced_accuracy', 'split4_test_balanced_accuracy',
       'mean_test_balanced_accuracy', 'std_test_balanced_accuracy',
       'rank_test_balanced_accuracy', 'split0_test_roc_auc',
       'split1_test_roc_auc', 'split2_test_roc_auc', 'split3_test_roc_auc',
       'split4_test_roc_auc', 'mean_test_roc_auc', 'std_test_roc_auc',
       'rank_test_roc_auc', 'split0_test_f1', 'split1_test_f1',
       'split2_test_f1', 

In [153]:
dfRes["classifier"] = dfRes["param_clf"].apply(lambda x: x.__str__().split("(")[0])
dfRes['site'] = dfRes['MainDataset'].apply(changeSite)
lColFirst = [
    "MainDataset",
    "Title",
    'site',
    "date",
    "classifier",
    'RunType',
    "cv",
    # 'FeatureSet',
    "brt_nTrue",
    "brt_nTop",
    'brt_md',
]
dfRes = MPutils.reorder_columns(dfRes, lColFirst)
dfRes.to_csv("OUTPUT/MP/05-classifiers/DataV4/CLFRunCombined.csv", index=False)
dfResOrg = deepcopy(dfRes)
lColsSel = lColFirst + [
    x
    for x in dfRes
    if (x.startswith("mean_test") or x.startswith("param_") or x.startswith("brt_"))
    and x not in lColFirst
]
dfRes[lColsSel].to_csv("OUTPUT/MP/05-classifiers/DataV4/CLFRunCombined_SelCol.csv")

# Go through Data and Simplify for inspection

In [154]:
dfRes = dfRes[lColsSel]

In [156]:
lSelMainData = [
"dfcmpltPreLC4",
"dfcmpltPreLC4_SITE1",
"dfcmpltPreLC4_SITE2",
"dfcmpltPreLC4_SITE3",
"dfcmpltPreLC4_SITE4",
"dfcmpltPreLC4_SITE5",
"dfcmpltPreLC4_SITE6",
"dfcmpltPreLC4_SITE7",
]

lPrettyCols =     [
        "MainDataset",
        "RunType",
        "classifier",
        "brt_nTrue",
        "brt_nTop",
        "accuracy",
        "balanced_accuracy",
        "roc_auc",
        "f1",
        "recall",
        "sensitivity",
        "specificity",
        "precision",
        "NPV",
        "PPV",
        "brt_md",
        'brt_params'
    ]
lPrettyColsSite = [
    "MainDataset",
    "RunType",
    'site',
    "classifier",
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    "NPV",
    "PPV",
]
s_brt = 'and (RunType=="Boruta_True" or RunType=="Boruta_Top")'
s_RF10 = 'classifier=="RandomForestClassifier" and param_clf__n_estimators==1000 and param_clf__max_depth==10'
s_RFXGB10 = 'param_clf__n_estimators==1000 and param_clf__max_depth==10'
s_CMPLT = 'MainDataset=="dfcmpltPreLC4"'
s_val = 'RunType=="HoldoutVal"'

In [157]:
dfRes = dfRes[dfRes['MainDataset'].isin(lSelMainData)]
dfRes.columns = [x.replace('mean_test_','') for x in dfRes.columns]

In [158]:
# FS_{tmpdataset}_Boruta_T1000_itrr500_th100_topR5_MD{md}.csv'
dfRes['brt_file'] = dfRes[['MainDataset','brt_md']].apply(lambda x: f"FS_{x['MainDataset']}_Boruta_T1000_itrr500_th100_topR5_MD{x['brt_md']:.0f}.csv",axis=1)

# Inspect Original Holdout Results
- May have duplicates
- DO NOT USE FOR RESULTS -- INTERNAL VALIDATION ONLY and COMPARISON ONLY

In [159]:
dfRes.query(f'{s_RF10} and RunType=="Holdout"').sort_values("balanced_accuracy", ascending=False)[lPrettyColsSite].T

,5,5,5,5,5,5,5,5,5
MainDataset,dfcmpltPreLC4_SITE4,dfcmpltPreLC4_SITE3,dfcmpltPreLC4_SITE3,dfcmpltPreLC4,dfcmpltPreLC4,dfcmpltPreLC4_SITE2,dfcmpltPreLC4_SITE2,dfcmpltPreLC4_SITE1,dfcmpltPreLC4_SITE1
RunType,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout
site,RIO,SAN,SAN,Global,Global,MTL,MTL,LDN,LDN
classifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier
accuracy,0.980645,0.832967,0.832967,0.697829,0.697829,0.703951,0.703951,0.613333,0.613333
balanced_accuracy,0.94,0.720303,0.720303,0.681052,0.681052,0.542991,0.542991,0.525,0.525
roc_auc,0.975385,0.895758,0.895758,0.776731,0.776731,0.710012,0.710012,0.6,0.6
f1,0.988819,0.897749,0.897749,0.604697,0.604697,0.187445,0.187445,0.719841,0.719841
recall,1.0,0.907273,0.907273,0.513734,0.513734,0.111429,0.111429,0.75,0.75
sensitivity,1.0,0.907273,0.907273,0.513734,0.513734,0.111429,0.111429,0.75,0.75


# Per site - Best Results, use this for validation

In [171]:
# SECTION - Determine the best Boruta and Top Boruta for each dataset
# dfTmp = dfRes.query(f'{s_RF10} and RunType=="Boruta_True"').sort_values('balanced_accuracy',ascending=False)
dfTmp = dfRes.query(f'RunType=="Boruta_True" and {s_RFXGB10}').sort_values('balanced_accuracy',ascending=False)

lTopBrt = []
for i in lSelMainData:
    if i in dfTmp['MainDataset'].values:
        dfTmpTop = dfTmp.query(f'MainDataset=="{i}"').sort_values('balanced_accuracy',ascending=False)
        lTopBrt.append(dfTmpTop.iloc[0,:])

In [169]:
print('Pretty Columns data first')
pd.DataFrame(lTopBrt)[lPrettyColsSite+['brt_md','brt_nTrue','Title','brt_file']].sort_values('balanced_accuracy',ascending=False)

Pretty Columns data first


,MainDataset,RunType,site,classifier,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_nTrue,Title,brt_file
87,dfcmpltPreLC4_SITE1,Boruta_True,LDN,RandomForestClassifier,0.860000,0.875000,0.850000,0.892857,0.850000,0.850000,0.900000,0.950000,0.733333,0.950000,3.0,4.0,dfcmpltPreLC4_SITE1_Boruta_True,FS_dfcmpltPreLC4_SITE1_Boruta_T1000_itrr500_th100_topR5_MD3.csv
91,dfcmpltPreLC4,Boruta_True,Global,RandomForestClassifier,0.723456,0.715487,0.789034,0.674343,0.639620,0.639620,0.791353,0.717731,0.730688,0.717731,3.0,21.0,dfcmpltPreLC4_Boruta_True,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_topR5_MD3.csv
169,dfcmpltPreLC4_SITE2,Boruta_True,MTL,RandomForestClassifier,0.664912,0.573712,0.601268,0.380357,0.333175,0.333175,0.814249,0.446023,0.730814,0.446023,5.0,2.0,dfcmpltPreLC4_SITE2_Boruta_True,FS_dfcmpltPreLC4_SITE2_Boruta_T1000_itrr500_th100_topR5_MD5.csv


In [170]:
print('Pretty Columns data last')
pd.DataFrame(lTopBrt)[['brt_md','brt_nTrue','Title','brt_file']+lPrettyColsSite].sort_values('MainDataset',ascending=True)

Pretty Columns data last


,brt_md,brt_nTrue,Title,brt_file,MainDataset,RunType,site,classifier,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV
91,3.0,21.0,dfcmpltPreLC4_Boruta_True,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_topR5_MD3.csv,dfcmpltPreLC4,Boruta_True,Global,RandomForestClassifier,0.723456,0.715487,0.789034,0.674343,0.639620,0.639620,0.791353,0.717731,0.730688,0.717731
87,3.0,4.0,dfcmpltPreLC4_SITE1_Boruta_True,FS_dfcmpltPreLC4_SITE1_Boruta_T1000_itrr500_th100_topR5_MD3.csv,dfcmpltPreLC4_SITE1,Boruta_True,LDN,RandomForestClassifier,0.860000,0.875000,0.850000,0.892857,0.850000,0.850000,0.900000,0.950000,0.733333,0.950000
169,5.0,2.0,dfcmpltPreLC4_SITE2_Boruta_True,FS_dfcmpltPreLC4_SITE2_Boruta_T1000_itrr500_th100_topR5_MD5.csv,dfcmpltPreLC4_SITE2,Boruta_True,MTL,RandomForestClassifier,0.664912,0.573712,0.601268,0.380357,0.333175,0.333175,0.814249,0.446023,0.730814,0.446023


In [166]:
s_query = f'{s_CMPLT} and param_clf__n_estimators==1000 and param_clf__max_depth==10 {s_brt}'
dfTmp = deepcopy(dfRes.query(s_query)).sort_values('balanced_accuracy',ascending=False)
print(s_query)
print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")
# print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")

# dfTmp[lPrettyCols]

MainDataset=="dfcmpltPreLC4" and param_clf__n_estimators==1000 and param_clf__max_depth==10 and (RunType=="Boruta_True" or RunType=="Boruta_Top")
Max balanced accuracy 0.7154865414781337


## Export -- save to file Preliminary Results

In [173]:
# # Create the ExcelWriter object
writer = pd.ExcelWriter('../OUTPUTS/DataV4/DataV4_prelim_preLongCOVID4_Brt_columns.xlsx', engine='xlsxwriter')
pd.DataFrame(lTopBrt)[lPrettyColsSite+['brt_md','brt_nTrue','Title','brt_file']].sort_values('balanced_accuracy',ascending=False).to_excel(writer,sheet_name='Results')

for i,d in pd.DataFrame(lTopBrt).iterrows():
    tmpdataset = d['MainDataset']
    md = int(d['brt_md'])
    f = f'OUTPUT/MP/05-classifiers/DataV4/{tmpdataset}/FS_{tmpdataset}_Boruta_T1000_itrr500_th100_topR5_MD{md}.csv'
    dfeat = pd.read_csv(f)
    MPutils.getTrueFeatList(f,asDF=True).to_excel(writer, sheet_name=d['site'])
    



#     # Save the Excel file
writer.close()


## Curious -- Difference between Boruta Selected Datasets

In [57]:
# lbrt_7 = MPutils.getTrueFeatList(
#     "OUTPUT/MP/05-classifiers/DataV4/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7.csv"
# )

# lbrt_5 = MPutils.getTrueFeatList(
#     "OUTPUT/MP/05-classifiers/DataV4/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD5.csv"
# )

# lbrt_3 = MPutils.getTrueFeatList(
#     "OUTPUT/MP/05-classifiers/DataV4/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD3.csv"
# )

# len(lbrt_7), len(lbrt_5), len(lbrt_3)

In [58]:
# set(lbrt_7).difference(set(lbrt_5))

In [59]:
# set(lbrt_5).difference(set(lbrt_7))

In [60]:
# set(lbrt_7).difference(set(lbrt_3))

In [61]:
# set(lbrt_3).difference(set(lbrt_5))

In [62]:
# set(lbrt_3).difference(set(lbrt_5))

In [63]:
# set(lbrt_5).difference(set(lbrt_3))

In [64]:
# set(lbrt_5).difference(set(lbrt_7))

# Validation Results

In [65]:
dfVal = dfRes.query(f'{s_RF10} and {s_val}').sort_values('balanced_accuracy',ascending=False)
dfVal

,MainDataset,Title,site,date,classifier,RunType,cv,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params


In [66]:
dfVal = dfVal.sort_values(['FeatureSet','MainDataset'])
dfVal

KeyError: 'FeatureSet'

In [67]:
sGlobal = 'FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_topR5_MD3'

dfVal['FeatureSetSpecific'] = dfVal['FeatureSet'].apply(lambda x:  'Global' if x == sGlobal else 'Site')

KeyError: 'FeatureSet'

In [75]:
# # Create the ExcelWriter object
writer = pd.ExcelWriter('../OUTPUTS/DataV4/DataV4_prelim_preLongCOVID4_Brt_columns.xlsx', engine='xlsxwriter')
pd.DataFrame(lTopBrt)[lPrettyColsSite+['brt_md','brt_nTrue','Title','brt_file']].sort_values('balanced_accuracy',ascending=False).to_excel(writer,sheet_name='Results')

for i,d in pd.DataFrame(lTopBrt).iterrows():
    tmpdataset = d['MainDataset']
    md = int(d['brt_md'])
    f = f'OUTPUT/MP/05-classifiers/DataV4/{tmpdataset}/FS_{tmpdataset}_Boruta_T1000_itrr500_th100_topR5_MD{md}.csv'
    dfeat = pd.read_csv(f)
    MPutils.getTrueFeatList(f,asDF=True).to_excel(writer, sheet_name=d['site'])
    
    dFeat = pd.read_csv(f'OUTPUT/MP/05-classifiers/DataV4/{tmpMainData}/FR_{tmpMainData}_HoldoutVal_{tmpFS}.csv')
    # f = f'OUTPUT/MP/05-classifiers/DataV4/{tmpdataset}/FS_{tmpdataset}_Boruta_T1000_itrr500_th100_topR5_MD{md}.csv'
    # dfeat = pd.read_csv(f)
    
    # MPutils.getTrueFeatList(f,asDF=True).to_excel(writer, sheet_name=d['site'])
    dFeat.to_excel(writer,sheet_name=sheetName)



#     # Save the Excel file
writer.close()


## EXTRA - Curious -- Difference between Boruta Selected Datasets

In [57]:
# lbrt_7 = MPutils.getTrueFeatList(
#     "OUTPUT/MP/05-classifiers/DataV4/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7.csv"
# )

# lbrt_5 = MPutils.getTrueFeatList(
#     "OUTPUT/MP/05-classifiers/DataV4/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD5.csv"
# )

# lbrt_3 = MPutils.getTrueFeatList(
#     "OUTPUT/MP/05-classifiers/DataV4/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD3.csv"
# )

# len(lbrt_7), len(lbrt_5), len(lbrt_3)

In [58]:
# set(lbrt_7).difference(set(lbrt_5))

In [59]:
# set(lbrt_5).difference(set(lbrt_7))

In [60]:
# set(lbrt_7).difference(set(lbrt_3))

In [61]:
# set(lbrt_3).difference(set(lbrt_5))

In [62]:
# set(lbrt_3).difference(set(lbrt_5))

In [63]:
# set(lbrt_5).difference(set(lbrt_3))

In [64]:
# set(lbrt_5).difference(set(lbrt_7))